# Análise de Discursos de Senadores da 56ª Legislatura (2019-2023)


Pipeline:
1) Coleta (JSON): lista de senadores, discursos por período/senador e texto integral do pronunciamento.
2) Limpeza e normalização.
3) Representação (TF‑IDF e embeddings).
4) Tópicos (clusterização + c-TF-IDF) e Sentimento (baseline).
5) Sumarização com RAG (esqueleto com citações).


In [1]:

# Dependências (descomente se precisar instalar)
%pip install requests pandas numpy scikit-learn nltk sentence-transformers umap-learn hdbscan bertopic unidecode beautifulsoup4 pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 13.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 43.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 48.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 27.2 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.40-cp312-cp312-linux_x86_64.whl size=4587717 sha256=81c18d229b1754c472f38e651268e

In [2]:

import os, re, time, datetime as dt
from typing import List, Dict, Any, Iterable
import requests, pandas as pd, numpy as np
from requests.adapters import HTTPAdapter, Retry

BASE = "https://legis.senado.leg.br/dadosabertos/"
OUT_DIR = "_data"; TEXT_DIR = "_textos"
os.makedirs(OUT_DIR, exist_ok=True); os.makedirs(TEXT_DIR, exist_ok=True)

def make_session() -> requests.Session:
    s = requests.Session()
    retries = Retry(total=8, backoff_factor=0.6, status_forcelist=[429,500,502,503,504], allowed_methods=["GET"])
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.headers.update({"Accept": "application/json"})
    return s

sess = make_session()

def yyyymmdd(d: dt.date) -> str: return d.strftime("%Y%m%d")
def make_windows(start: dt.date, end: dt.date, days_per_window: int = 31):
    cur = start; one = dt.timedelta(days=1)
    while cur <= end:
        w_end = min(cur + dt.timedelta(days=days_per_window - 1), end)
        yield (cur, w_end); cur = w_end + one

def safe_root_dict(j: Dict[str, Any]) -> Dict[str, Any]:
    for k, v in j.items():
        if isinstance(v, dict): return v
    return j

def extract_pronunciamentos(obj: Any) -> List[Dict[str, Any]]:
    out = []
    def rec(x):
        if isinstance(x, dict):
            for k, v in x.items():
                if isinstance(k,str) and k.lower()=="pronunciamento" and isinstance(v, list): out.extend(v)
                else: rec(v)
        elif isinstance(x, list):
            for it in x: rec(it)
    rec(obj); return out


In [4]:

def fetch_senadores_legislatura(legislatura: int) -> pd.DataFrame:
    url = f"{BASE}senador/lista/legislatura/{legislatura}.json"
    r = sess.get(url, params={"v":4}, timeout=90); r.raise_for_status()
    root = safe_root_dict(r.json())
    parls = (root.get("Parlamentares") or {}).get("Parlamentar") or []
    df = pd.json_normalize(parls, sep=".")
    for c in list(df.columns):
        if c.lower().endswith("codigoparlamentar"):
            df = df.rename(columns={c: "CodigoParlamentar"}); break
    df["CodigoParlamentar"] = df["CodigoParlamentar"].astype(str).str.strip()
    return df

LEGISLATURA = 56
df_sen = fetch_senadores_legislatura(LEGISLATURA)
df_sen.to_csv(os.path.join(OUT_DIR, f"senadores_leg{LEGISLATURA}_lista.csv"), index=False, sep=";", encoding="utf-8-sig")
df_sen.head()


,CodigoParlamentar,IdentificacaoParlamentar.NomeParlamentar,IdentificacaoParlamentar.NomeCompletoParlamentar,IdentificacaoParlamentar.SexoParlamentar,IdentificacaoParlamentar.FormaTratamento,IdentificacaoParlamentar.SiglaPartidoParlamentar,Mandatos.Mandato,IdentificacaoParlamentar.CodigoPublicoNaLegAtual,IdentificacaoParlamentar.UrlFotoParlamentar,IdentificacaoParlamentar.UrlPaginaParlamentar,IdentificacaoParlamentar.UrlPaginaParticular,IdentificacaoParlamentar.EmailParlamentar,IdentificacaoParlamentar.UfParlamentar
0,5573,Abel Rebouças,Abel Rebouças São José,Masculino,Senador,PDT,"[{'CodigoMandato': '492', 'UfParlamentar': 'BA...",NaN,NaN,NaN,NaN,NaN,NaN
1,4981,Acir Gurgacz,Acir Marcos Gurgacz,Masculino,Senador,PDT,"[{'CodigoMandato': '515', 'UfParlamentar': 'RO...",916,http://www.senado.leg.br/senadores/img/fotos-o...,http://www25.senado.leg.br/web/senadores/senad...,https://acirgurgacz.com.br/,sen.acirgurgacz@senado.leg.br,NaN
2,5918,Adilson Gomes,Adilson Gomes Silva,Masculino,Senador,NaN,"[{'CodigoMandato': '526', 'UfParlamentar': 'PE...",NaN,NaN,NaN,NaN,NaN,NaN
3,5625,Adilson Silva dos Santos,Adilson Silva dos Santos,Masculino,Senador,PEN,"[{'CodigoMandato': '497', 'UfParlamentar': 'RS...",NaN,NaN,NaN,NaN,NaN,NaN
4,6026,Afonso Parente,Afonso Valter Parente Pinto,Masculino,Senador,NaN,"[{'CodigoMandato': '578', 'UfParlamentar': 'RR...",NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

def fetch_discursos_senador(codigo: str, data_inicio: dt.date, data_fim: dt.date) -> pd.DataFrame:
    frames = []
    for ini, fim in make_windows(data_inicio, data_fim, 31):
        url = f"{BASE}senador/{codigo}/discursos.json"
        params = {"dataInicio": yyyymmdd(ini), "dataFim": yyyymmdd(fim)}
        r = sess.get(url, params=params, timeout=90); r.raise_for_status()
        pron = extract_pronunciamentos(r.json())
        if not pron: continue
        df = pd.json_normalize(pron, sep="."); df["CodigoParlamentar"] = str(codigo)
        frames.append(df)
    return pd.concat(frames, ignore_index=True, sort=False) if frames else pd.DataFrame()

INI = dt.date(2025,1,1); FIM = dt.date(2025,9,16)
codigos_exemplo = df_sen["CodigoParlamentar"].astype(str).head(5).tolist()
dfs = [fetch_discursos_senador(c, INI, FIM) for c in codigos_exemplo]
df_disc = pd.concat([d for d in dfs if not d.empty], ignore_index=True, sort=False) if dfs else pd.DataFrame()
df_disc.to_csv(os.path.join(OUT_DIR, f"discursos_amostra_leg{LEGISLATURA}_{INI}_{FIM}.csv"), index=False, sep=";", encoding="utf-8-sig")
df_disc.head()


ValueError: No objects to concatenate

In [ ]:

import re
def fetch_texto_integral(codigo_pronunciamento: str) -> str:
    url = f"{BASE}discurso/texto-integral/{codigo_pronunciamento}.txt"
    r = sess.get(url, headers={"Accept":"text/plain"}, timeout=90)
    if r.status_code == 404: return ""
    r.raise_for_status()
    txt = re.sub(r"\s+\n", "\n", r.text); txt = re.sub(r"[ \t]+", " ", txt).strip()
    return txt

def anexar_textos(df_disc: pd.DataFrame, salvar_arquivos: bool=True, pasta: str="_textos") -> pd.DataFrame:
    if "CodigoPronunciamento" not in df_disc.columns:
        cand = [c for c in df_disc.columns if c.lower().endswith("codigopronunciamento")]
        if cand: df_disc = df_disc.rename(columns={cand[0]: "CodigoPronunciamento"})
        else: raise KeyError("Coluna CodigoPronunciamento não encontrada.")
    os.makedirs(pasta, exist_ok=True)
    paths = []
    for cod in df_disc["CodigoPronunciamento"].astype(str):
        try:
            txt = fetch_texto_integral(cod); path = os.path.join(pasta, f"{cod}.txt")
            with open(path, "w", encoding="utf-8") as f: f.write(txt)
            paths.append(path)
        except Exception as e:
            print(f"[TEXTO ERRO] {cod}: {e}"); paths.append("")
    df_disc["ArquivoTextoIntegral"] = paths
    return df_disc

if not df_disc.empty:
    df_disc = anexar_textos(df_disc, True, TEXT_DIR)
    df_disc.to_csv(os.path.join(OUT_DIR, f"discursos_amostra_leg{LEGISLATURA}_{INI}_{FIM}_com_texto.csv"), index=False, sep=";", encoding="utf-8-sig")
df_disc.head()


### (Opcional) TF‑IDF, embeddings e tópicos — preencha conforme sua necessidade